<a href="https://colab.research.google.com/github/KayvanShah1/usc-dsci553-data-mining-sp24/blob/main/assignment-2/notebooks/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup & Installation

In [1]:
!pip install pyspark ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=90e29d51db783940b0dcea4e552cd6ff0790040b250bad76bc148b79e9571736
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
%%bash
java --version

openjdk 11.0.21 2023-10-17
OpenJDK Runtime Environment (build 11.0.21+9-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.21+9-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)


# Imports

In [3]:
import os
import sys
import json
from pyspark import SparkContext

from pprint import pprint

%load_ext autotime

time: 276 µs (started: 2024-02-25 08:15:04 +00:00)


In [4]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/DSCI553/hw2")
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/DSCI553/hw2'

time: 172 ms (started: 2024-02-25 08:15:04 +00:00)


In [5]:
!ls

Assignment_2.ipynb  small2.csv	      t1-s2-c1.txt	t1-s2-c2.txt		       task1_ref.py
small1.csv	    t1-s2-c1-ref.txt  t1-s2-c2-ref.txt	ta_feng_all_months_merged.csv
time: 306 ms (started: 2024-02-25 08:15:05 +00:00)


# Tasks

In [15]:
def compare_files(file1_path, file2_path):
    # Read the contents of the first file
    with open(file1_path, 'r') as file1:
        file1_contents = file1.read().strip()

    # Read the contents of the second file
    with open(file2_path, 'r') as file2:
        file2_contents = file2.read().strip()

    # Compare the contents of the files
    if file1_contents == file2_contents:
        print("The contents of the files are the same.")
    else:
        print("The contents of the files are different.")

time: 878 µs (started: 2024-02-25 08:53:22 +00:00)


## Task 1

In [52]:
import sys
import time
from pyspark import SparkContext, SparkConf

from collections import Counter
from itertools import combinations, chain


def generate_baskets(data, case):
    """
    Generate baskets based on the specified case.
    """
    if case == 1:
        # Group Business
        baskets = data.map(lambda line: line.strip().split(",")).map(lambda x: (x[0], [x[1]]))
    elif case == 2:
        # Group User
        baskets = data.map(lambda line: line.strip().split(",")).map(lambda x: (x[1], [x[0]]))
    return baskets.reduceByKey(lambda a, b: a + b)


def hash_function(item1, item2, num_buckets):
    """
    Simple hash function to distribute items into different buckets.
    """
    hash_val = int(item1) ^ int(item2)
    return hash_val % num_buckets


def PCY(baskets, total_num_baskets, support, num_buckets):
    """
    Run the PCY algorithm to find candidates
    """
    candidates = []
    baskets = list(baskets)

    # Calculate the support ratio based on the number of baskets in the current partition and the total number of baskets
    support_ratio = len(baskets) / total_num_baskets

    # Calculate the partition support threshold by multiplying the support ratio with the desired support
    partition_support_threshold = support_ratio * support

    # First pass:
    # Initialize counters to keep track of item counts, and hashes
    item_counts = Counter()
    hash_counts = Counter()

    # Initialize a bitmap list with zeros, representing whether each hash bucket meets the support threshold
    bitmap = [0] * num_buckets

    for basket in baskets:
        # Count item pairs
        item_counts.update(basket)

        pairs = list(combinations(basket, 2))

        # Hash pairs and count the number of occurence every hash
        hashes = [hash_function(item[0], item[1], num_buckets) for item in pairs]
        hash_counts.update(hashes)

    # Update the bitmap if count exceeds support threshold
    for h, count in hash_counts.items():
        if count >= partition_support_threshold:
            bitmap[h] = 1

    # Second Pass:
    frequent_singles = []
    for item, count in item_counts.items():
        if count >= partition_support_threshold:
            frequent_singles.append(item)
            candidates.append((tuple([item]), 1))

    frequent_pairs = []
    for item in combinations(frequent_singles, 2):
        hash_val = hash_function(item[0], item[1], num_buckets)
        if bitmap[hash_val] == 1:
            frequent_pairs.append(item)

    if len(frequent_pairs) == 0:
        return []

    # Filter items from basket that are not frequent
    baskets = [[item for item in basket if item in frequent_singles] for basket in baskets]

    # Initialize candidate dictionary
    candidate_dict = {pair: 0 for pair in frequent_pairs}

    # Count occurrences of frequent pairs in baskets
    for basket in baskets:
        for pair in frequent_pairs:
            if set(pair).issubset(basket):
                candidate_dict[pair] += 1

    # Filter the pair that do not meet the support threshold
    candidate_dict = {
        pair: count for pair, count in candidate_dict.items() if count >= partition_support_threshold
    }

    # Append new candidates to the candidates list
    candidates += [(pair, 1) for pair in candidate_dict.keys()]

    # Valid Candidates
    val_candidates = candidate_dict.keys()

    # Check for frequent pairs in larger sets
    k = 3
    while len(val_candidates) > 0:
        unq_singles = set(chain.from_iterable(val_candidates))
        itemsets = list(combinations(unq_singles, k))

        # Initialize candidate dictionary
        candidate_dict = {itemset: 0 for itemset in itemsets}

        # Count occurrences of frequent pairs in baskets
        for basket in baskets:
            for itemset in itemsets:
                # Check if the candidate itemset is a subset of the current basket
                if set(itemset).issubset(basket):
                    candidate_dict[itemset] += 1

        # Filter the pair that do not meet the support threshold
        candidate_dict = {
            itemset: count for itemset, count in candidate_dict.items() if count >= partition_support_threshold
        }

        # Append new candidates to the candidates list
        candidates += [(itemset, 1) for itemset in candidate_dict.keys()]

        # Update Valid Candidates
        val_candidates = candidate_dict.keys()

        # Update Order of larger itemset for next iteration
        k += 1

    return candidates


def check_itemsets_in_basket(baskets, candidates):
    """
    Check if candidate itemsets are frequent in the given baskets.

    Parameters:
        baskets (iterable): An iterable containing baskets of items.
        candidates (iterable): An iterable containing candidate itemsets.

    Returns:
        frequent_itemsets (list): A list of frequent itemsets found in the baskets.
    """
    candidates = list(candidates)
    frequent_itemsets = []

    for basket in baskets:
        for itemset in candidates:
            # Check if the candidate itemset is a subset of the current basket
            if set(itemset).issubset(basket):
                frequent_itemsets.append(itemset)

    return frequent_itemsets


def SON(baskets, support, num_buckets=1000):
    """
    Perform SON algorithm to find frequent itemsets.

    Args:
        baskets (RDD): RDD containing baskets of items.
        support (int): Minimum support threshold.
        num_buckets (int, optional): Number of buckets for the PCY algorithm. Defaults to 1000.

    Returns:
        tuple: A tuple containing two dictionaries:
            - candidates_map: A dictionary mapping length of itemsets to the list of frequent candidates.
            - frequent_itemsets: A dictionary mapping length of frequent itemsets to the list of frequent itemsets.
    """
    # Count the number of baskets
    num_baskets = baskets.count()

    # Extract individual items from baskets
    items = baskets.values()

    # Stage 1: Finding Frequent Itemsets using PCY
    candidates = (
        items.mapPartitions(lambda chunk: PCY(chunk, num_baskets, support, num_buckets))
        .reduceByKey(lambda x, y: x + y)
        .map(lambda x: tuple(sorted(x[0])))
        .distinct()
        .sortBy(lambda x: (len(x), x))
    )

    # Group candidates by length and collect as a map
    candidates_map = (
        candidates
        .groupBy(lambda x: len(x))
        .mapValues(list)  # Convert grouped values to a list
        .sortByKey()
        .collectAsMap()
    )

    # Retrieve frequent candidates as a list
    candidates = candidates.collect()

    # Stage 2: Candidate Pruning
    frequent_itemsets = (
        items.mapPartitions(lambda chunk: check_itemsets_in_basket(chunk, candidates))
        .map(lambda itemset: (itemset, 1))
        .reduceByKey(lambda x, y: x + y)
        .filter(lambda x: x[1] >= support)
        .map(lambda x: tuple(sorted(x[0])))
        .sortBy(lambda x: (len(x), x))
        .groupBy(lambda x: len(x))
        .mapValues(list)  # Convert grouped values to a list
        .sortByKey()
        .collectAsMap()
    )
    return candidates_map, frequent_itemsets


def task1(case_number, support, input_file_path, output_file_path):
    # Initialize Spark
    conf = SparkConf().setAppName("Task 1")
    spark = SparkContext(conf=conf).getOrCreate()
    spark.setLogLevel("ERROR")

    try:
        start_time = time.time()
        # Read the input data
        data = spark.textFile(input_file_path)

        # Drop the header row
        data = data.filter(lambda row: row != 'user_id,business_id')

        # Generate baskets based on the specified case
        baskets = generate_baskets(data, case_number).cache()

        # Run the SON Algorithm
        candidates, frequent_itemsets = SON(baskets, support, num_buckets=1000)

        # Output the results
        with open(output_file_path, "w") as f:
            f.write(f"Candidates:\n")
            for _, itemsets in candidates.items():
                itemsets = ",".join(map(str, itemsets)).replace(",)", ")")
                f.write(f"{itemsets}\n\n")
            f.write("Frequent Itemsets:\n")
            for line, itemsets in frequent_itemsets.items():
                itemsets = ",".join(map(str, itemsets)).replace(",)", ")")
                f.write(f"{itemsets}")
                if line < len(frequent_itemsets):
                    f.write("\n\n")

        execution_time = time.time() - start_time
        print(f"Duration: {execution_time}\n")

    finally:
        # Stop Spark
        spark.stop()


# if __name__ == "__main__":
#     if len(sys.argv) != 5:
#         print(
#             "Usage: spark-submit --executor-memory 4G --driver-memory 4G "
#             "task1.py <case_number> <support> <input_file_path> <output_file_path>"
#         )
#         sys.exit(1)

#     # Read input parameters
#     case_number = int(sys.argv[1])
#     support = int(sys.argv[2])
#     input_file_path = sys.argv[3]
#     output_file_path = sys.argv[4]

#     task1(case_number, support, input_file_path, output_file_path)


task1(2, 9, "small2.csv", "t1-s2-c2.txt")
task1(1, 4, "small2.csv", "t1-s2-c1.txt")

Duration: 22.17356586456299

Duration: 33.39101719856262

time: 57.7 s (started: 2024-02-25 10:54:18 +00:00)


### Ref

In [27]:
# @title Reference Code Task 1 { vertical-output: true, form-width: "50%" }
%%writefile task1_ref.py
from pyspark import SparkContext
from itertools import combinations as comb
import time
import sys
import copy

def hash(x, y, n_bucket):
  return (x ^ y) % n_bucket

def group_bus(lines):
    return lines.map(lambda row: (row[0], row[1])).groupByKey().mapValues(set).map(lambda row: (row[0], list(row[1])))

def group_user(lines):
    return lines.map(lambda row: (row[1], row[0])).groupByKey().mapValues(set).map(lambda row: (row[0], list(row[1])))

def PCY(baskets, total_n, support, n_bucket):
    baskets = list(baskets)
    item_cnt = {}
    hash_cnt = {}
    bitmap=[0 for x in range(0, n_bucket)]
    candidates = []
    p = len(baskets) / total_n
    t = p * support

    #PCY pass1
    for basket in baskets:
        for item in basket:
            if item not in item_cnt.keys():
                item_cnt[item] = 1
            else:
                item_cnt[item] += 1
        for i in range(0, len(basket) - 1):
            for j in range(i + 1, len(basket)):
                idx = hash(int(basket[i]), int(basket[j]), n_bucket)
                if idx not in hash_cnt.keys():
                    hash_cnt[idx] = 1
                else:
                    hash_cnt[idx] += 1
        for key, value in hash_cnt.items():
            if value >= t:
                bitmap[key] = 1

    #PCY pass2
    single_freq = []
    for key, value in item_cnt.items():
        if value >= t:
            single_freq.append(key)
            candidates.append((tuple([key]), 1))
    #single_freq = sorted(single_freq)
    #print(candidates)
    pair_freq = []
    for i in range(0, len(single_freq) - 1):
        for j in range(i + 1, len(single_freq)):
            pair_freq.append((single_freq[i], single_freq[j]))
    for pair in pair_freq:
        if bitmap[hash(int(pair[0]), int(pair[1]), n_bucket)] != 1:
            pair_freq.remove(pair)

    #print(candidates)
    if len(pair_freq) == 0:
        return []
    else:
        baskets_temp = []
        for basket in baskets:
            basket_1 = copy.deepcopy(basket)
            for item in basket:
                if item not in single_freq:
                    basket_1.remove(item)
            baskets_temp.append(basket_1)
        baskets = baskets_temp
        #pair_freq = sorted(pair_freq)
        #print(pair_freq)
        cnt_dict = dict(zip(pair_freq, [0] * len(pair_freq)))
        for basket in baskets:
            for key in cnt_dict.keys():
                if set(list(key)) <= set(basket):
                    cnt_dict[key] += 1
        for key, value in cnt_dict.copy().items():
            if value < t:
                del cnt_dict[key]
            else:
                candidates.append((tuple(key), 1))
        candidate = cnt_dict.keys()
        #print(candidate)
        k = 3
        while(True):
            #candidate = sorted(candidate)
            temp = list(comb(set(a for b in candidate for a in b), k))
            cnt_dict = dict(zip(temp, [0] * len(temp)))
            for basket in baskets:
                for key in cnt_dict.keys():
                    #print(set(list(key)), set(basket))
                    if set(list(key)) <= set(basket):
                        cnt_dict[key] += 1
            #print(cnt_dict, t)
            for key, value in cnt_dict.copy().items():
                if value < t:
                    del cnt_dict[key]
                else:
                    candidates.append((tuple(key), 1))
            candidate = cnt_dict.keys()
            if len(candidate) > 0:
                k += 1
            else:
                break
        #print(candidates)
        return candidates

def SON(baskets, candidates):
    baskets = list(baskets)
    result = {}
    for basket in baskets:
        for items in candidates:
            flag = 0
            for item in items:
                if item not in basket:
                    flag = 1
                    break
            if flag == 0:
                idx = tuple(items)
                if idx not in result.keys():
                    result[idx] = 1
                else:
                    result[idx] += 1
    return result.items()

if __name__ == '__main__':
    case_n = sys.argv[1]
    support = sys.argv[2]
    input_path = sys.argv[3]
    output_path = sys.argv[4]

    n_bucket = 1000
    spark = SparkContext(appName= "task1")
    t_s = time.time()
    #spark = SparkSession.builder.enableHiveSupport().getOrCreate()
    #lines = spark.sparkContext.textFile(input_path)
    lines = spark.textFile(input_path)
    #print(lines.collect())
    lines = lines.filter(lambda row: row != 'user_id,business_id').map(lambda row: row.split(','))

    #Case1 or 2
    if int(case_n) == 1:
        baskets = group_bus(lines)
    elif int(case_n) == 2:
        baskets = group_user(lines)

    #SON algorithm stage #1 -> PCY
    #n_partitions = 5
    total_n = baskets.count()
    #baskets = baskets.partitionBy(n_partitions)
    p = baskets.getNumPartitions()
    items = baskets.values()
    a = items.collect()
    candidates = items.mapPartitions(lambda baskets: PCY(baskets, total_n, float(support), n_bucket)).reduceByKey(lambda x, y: x + y).collect()
    #print(candidates)
    result_temp = []
    for i in range(0, len(candidates)):
        result_temp.append(sorted(candidates[i][0]))
    result_temp = sorted(result_temp)
    result_temp = sorted(result_temp, key = lambda x: len(x))
    last = result_temp[-1]
    for i in range(len(result_temp) - 2, -1, -1):
        if last == result_temp[i]:
            del result_temp[i]
        else:
            last = result_temp[i]
    #print(result_temp)
    #Output candidates
    result = 'Candidates:\n'
    l = 1
    for i in range(0, len(result_temp)):
        if len(result_temp[i]) == 1:
            result = result + '(' + str(result_temp[i])[1:-1] + '),'
        else:
            if len(result_temp[i]) != l:
                result = result[:-1] + "\n\n" + '(' + str(result_temp[i])[1:-1] + "),"
                l = len(result_temp[i])
            else:
                result = result + '(' + str(result_temp[i])[1:-1] + "),"
    result = result[:-1] + '\n\n'
    #print(result)

    #SON algorithm stage #2
    freq_items = items.mapPartitions(lambda baskets: SON(baskets, result_temp)).reduceByKey(lambda x, y: x + y).filter(lambda x : x[1] >= int(support)).collect()
    result_temp = []
    for i in range(0, len(freq_items)):
        result_temp.append(sorted(freq_items[i][0]))
    result_temp = sorted(result_temp)
    result_temp = sorted(result_temp, key = lambda x: len(x))
    last = result_temp[-1]
    for i in range(len(result_temp) - 2, -1, -1):
        if last == result_temp[i]:
            del result_temp[i]
        else:
            last = result_temp[i]
    #print(result_temp)
    result += 'Frequent Itemsets:\n'
    l = 1
    for i in range(0, len(result_temp)):
        if len(result_temp[i]) == 1:
            result = result + '(' + str(result_temp[i])[1:-1] + '),'
        else:
            if len(result_temp[i]) != l:
                result = result[:-1] + "\n\n" + '(' + str(result_temp[i])[1:-1] + "),"
                l = len(result_temp[i])
            else:
                result = result + '(' + str(result_temp[i])[1:-1] + "),"
    result = result[:-1]

    with open(output_path, 'w+') as f:
        f.write(result)

    t_e = time.time()

    print("Duration: ", t_e - t_s)

Overwriting task1_ref.py
time: 16.9 ms (started: 2024-02-25 04:28:23 +00:00)


In [31]:
!spark-submit task1_ref.py 2 9 small2.csv t1-s2-c2-ref.txt --executor-memory 4G --driver-memory 4G

24/02/25 04:36:04 INFO SparkContext: Running Spark version 3.5.0
24/02/25 04:36:04 INFO SparkContext: OS info Linux, 6.1.58+, amd64
24/02/25 04:36:04 INFO SparkContext: Java version 11.0.21
24/02/25 04:36:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/25 04:36:05 INFO ResourceUtils: ==============================================================
24/02/25 04:36:05 INFO ResourceUtils: No custom resources configured for spark.driver.
24/02/25 04:36:05 INFO ResourceUtils: ==============================================================
24/02/25 04:36:05 INFO SparkContext: Submitted application: task1
24/02/25 04:36:05 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount

## Compare files

In [53]:
compare_files("t1-s2-c2-ref.txt", "t1-s2-c2.txt")
compare_files("t1-s2-c1-ref.txt", "t1-s2-c1.txt")

The contents of the files are the same.
The contents of the files are the same.
time: 18.5 ms (started: 2024-02-25 10:55:23 +00:00)


## Task 2

In [58]:
import sys
import time
from pyspark import SparkContext, SparkConf

from collections import Counter
from itertools import combinations, chain


def generate_baskets(data, filter_threshold):
    """
    Generate baskets based on the specified case.
    """
    baskets = (
        data.map(lambda x: (x[0], [x[1]]))
        .reduceByKey(lambda a, b: a + b)
        .filter(lambda x: len(x[1]) > filter_threshold)
    )
    return baskets


def hash_function(item1, item2, num_buckets):
    """
    Simple hash function to distribute items into different buckets.
    """
    hash_val = int(item1) ^ int(item2)
    return hash_val % num_buckets


def PCY(baskets, total_num_baskets, support, num_buckets):
    """
    Run the PCY algorithm to find candidates
    """
    candidates = []
    baskets = list(baskets)

    # Calculate the support ratio based on the number of baskets in the current partition and the total number of baskets
    support_ratio = len(baskets) / total_num_baskets

    # Calculate the partition support threshold by multiplying the support ratio with the desired support
    partition_support_threshold = support_ratio * support

    # First pass:
    # Initialize counters to keep track of item counts, and hashes
    item_counts = Counter()
    hash_counts = Counter()

    # Initialize a bitmap list with zeros, representing whether each hash bucket meets the support threshold
    bitmap = [0] * num_buckets

    for basket in baskets:
        # Count item pairs
        item_counts.update(basket)

        pairs = list(combinations(basket, 2))

        # Hash pairs and count the number of occurence every hash
        hashes = [hash_function(item[0], item[1], num_buckets) for item in pairs]
        hash_counts.update(hashes)

    # Update the bitmap if count exceeds support threshold
    for h, count in hash_counts.items():
        if count >= partition_support_threshold:
            bitmap[h] = 1

    # Second Pass:
    frequent_singles = []
    for item, count in item_counts.items():
        if count >= partition_support_threshold:
            frequent_singles.append(item)
            candidates.append((tuple([item]), 1))

    frequent_pairs = []
    for item in combinations(frequent_singles, 2):
        hash_val = hash_function(item[0], item[1], num_buckets)
        if bitmap[hash_val] == 1:
            frequent_pairs.append(item)

    if len(frequent_pairs) == 0:
        return []

    # Filter items from basket that are not frequent
    baskets = [[item for item in basket if item in frequent_singles] for basket in baskets]

    # Initialize candidate dictionary
    candidate_dict = {pair: 0 for pair in frequent_pairs}

    # Count occurrences of frequent pairs in baskets
    for basket in baskets:
        for pair in frequent_pairs:
            if set(pair).issubset(basket):
                candidate_dict[pair] += 1

    # Filter the pair that do not meet the support threshold
    candidate_dict = {
        pair: count for pair, count in candidate_dict.items() if count >= partition_support_threshold
    }

    # Append new candidates to the candidates list
    candidates += [(pair, 1) for pair in candidate_dict.keys()]

    # Valid Candidates
    val_candidates = candidate_dict.keys()

    # Check for frequent pairs in larger sets
    k = 3
    while len(val_candidates) > 0:
        unq_singles = set(chain.from_iterable(val_candidates))
        itemsets = list(combinations(unq_singles, k))

        # Initialize candidate dictionary
        candidate_dict = {itemset: 0 for itemset in itemsets}

        # Count occurrences of frequent pairs in baskets
        for basket in baskets:
            for itemset in itemsets:
                # Check if the candidate itemset is a subset of the current basket
                if set(itemset).issubset(basket):
                    candidate_dict[itemset] += 1

        # Filter the pair that do not meet the support threshold
        candidate_dict = {
            itemset: count for itemset, count in candidate_dict.items() if count >= partition_support_threshold
        }

        # Append new candidates to the candidates list
        candidates += [(itemset, 1) for itemset in candidate_dict.keys()]

        # Update Valid Candidates
        val_candidates = candidate_dict.keys()

        # Update Order of larger itemset for next iteration
        k += 1

    return candidates


def check_itemsets_in_basket(baskets, candidates):
    """
    Check if candidate itemsets are frequent in the given baskets.

    Parameters:
        baskets (iterable): An iterable containing baskets of items.
        candidates (iterable): An iterable containing candidate itemsets.

    Returns:
        frequent_itemsets (list): A list of frequent itemsets found in the baskets.
    """
    candidates = list(candidates)
    frequent_itemsets = []

    for basket in baskets:
        for itemset in candidates:
            # Check if the candidate itemset is a subset of the current basket
            if set(itemset).issubset(basket):
                frequent_itemsets.append(itemset)

    return frequent_itemsets


def SON(baskets, support, num_buckets=1000):
    """
    Perform SON algorithm to find frequent itemsets.

    Args:
        baskets (RDD): RDD containing baskets of items.
        support (int): Minimum support threshold.
        num_buckets (int, optional): Number of buckets for the PCY algorithm. Defaults to 1000.

    Returns:
        tuple: A tuple containing two dictionaries:
            - candidates_map: A dictionary mapping length of itemsets to the list of frequent candidates.
            - frequent_itemsets: A dictionary mapping length of frequent itemsets to the list of frequent itemsets.
    """
    # Count the number of baskets
    num_baskets = baskets.count()

    # Extract individual items from baskets
    items = baskets.values()

    # Stage 1: Finding Frequent Itemsets using PCY
    candidates = (
        items.mapPartitions(lambda chunk: PCY(chunk, num_baskets, support, num_buckets))
        .reduceByKey(lambda x, y: x + y)
        .map(lambda x: tuple(sorted(x[0])))
        .distinct()
        .sortBy(lambda x: (len(x), x))
    )

    # Group candidates by length and collect as a map
    candidates_map = (
        candidates
        .groupBy(lambda x: len(x))
        .mapValues(list)  # Convert grouped values to a list
        .sortByKey()
        .collectAsMap()
    )

    # Retrieve frequent candidates as a list
    candidates = candidates.collect()

    # Stage 2: Candidate Pruning
    frequent_itemsets = (
        items.mapPartitions(lambda chunk: check_itemsets_in_basket(chunk, candidates))
        .map(lambda itemset: (itemset, 1))
        .reduceByKey(lambda x, y: x + y)
        .filter(lambda x: x[1] >= support)
        .map(lambda x: tuple(sorted(x[0])))
        .sortBy(lambda x: (len(x), x))
        .groupBy(lambda x: len(x))
        .mapValues(list)  # Convert grouped values to a list
        .sortByKey()
        .collectAsMap()
    )
    return candidates_map, frequent_itemsets


def create_date_customer_id(date, customer_id):
    date = date.replace('"',"").split("/")
    date = f"{date[0]}/{date[1]}/{date[2][2:]}"
    customer_id = int(customer_id.replace('"',""))
    return f"{date}-{customer_id}"


def process_prod_id(prod_id):
    prod_id = int(prod_id.replace('"',""))
    return f"{prod_id}"


def preprocess_data(data):
    # Drop the header row
    data_header = data.first()
    data = (
        data.filter(lambda row: row != data_header)
        .map(lambda row: row.split(","))
        .map(lambda row: [create_date_customer_id(row[0], row[1]), process_prod_id(row[5])])
    )
    return data


def task2(filter_threshold, support, input_file_path, output_file_path):
    # Initialize Spark
    conf = SparkConf().setAppName("Task 2")
    spark = SparkContext(conf=conf).getOrCreate()
    spark.setLogLevel("ERROR")

    try:
        start_time = time.time()

        # Read the input data
        data = spark.textFile(input_file_path)
        data = preprocess_data(data)

        # Generate baskets based on the specified filter threshold
        baskets = generate_baskets(data, filter_threshold).cache()

        # Run the SON Algorithm
        candidates, frequent_itemsets = SON(baskets, support, num_buckets=1000)

        # Output the results
        with open(output_file_path, "w") as f:
            f.write(f"Candidates:\n")
            for _, itemsets in candidates.items():
                itemsets = ",".join(map(str, itemsets)).replace(",)", ")")
                f.write(f"{itemsets}\n\n")
            f.write("Frequent Itemsets:\n")
            for line, itemsets in frequent_itemsets.items():
                itemsets = ",".join(map(str, itemsets)).replace(",)", ")")
                f.write(f"{itemsets}")
                if line < len(frequent_itemsets):
                    f.write("\n\n")

        execution_time = time.time() - start_time
        print(f"Duration: {execution_time}\n")

    finally:
        # Stop Spark
        spark.stop()


# if __name__ == "__main__":
#     if len(sys.argv) != 5:
#         print(
#             "Usage: spark-submit --executor-memory 4G --driver-memory 4G "
#             "task2.py <filter_threshold> <support> <input_file_path> <output_file_path>"
#         )
#         sys.exit(1)

#     # Read input parameters
#     filter_threshold = int(sys.argv[1])
#     support = int(sys.argv[2])
#     input_file_path = sys.argv[3]
#     output_file_path = sys.argv[4]

#     task2(filter_threshold, support, input_file_path, output_file_path)


task2(20, 5, "ta_feng_all_months_merged.csv", "t2.txt")

Duration: 8.921281814575195

time: 10.3 s (started: 2024-02-25 11:05:01 +00:00)


# THE END